In [ ]:
import psycopg2
import csv

here i connect to db and create a new bd after this i load data wich i got from main(parser) i have another file which take data from this bd and convert it in json this json i send in server.js but you can uncommite code in server.js and after connecting to db,launch server.js from browser.

In [ ]:
# Database connection parameters
DB_NAME = "movies_db"  # Change to your preferred database name
USER = "postgres"
PASSWORD = "tupakill3"
HOST = "localhost"
PORT = "5432"

# Connect to PostgreSQL (without specifying a database) to create the DB first
conn = psycopg2.connect(
    dbname="postgres", user=USER, password=PASSWORD, host=HOST, port=PORT
)
conn.autocommit = True
cur = conn.cursor()

# Create the database if it does not exist
cur.execute(f"SELECT 1 FROM pg_database WHERE datname = '{DB_NAME}';")
exists = cur.fetchone()
if not exists:
    cur.execute(f"CREATE DATABASE {DB_NAME};")
    print(f"Database '{DB_NAME}' created successfully.")
else:
    print(f"Database '{DB_NAME}' already exists.")

cur.close()
conn.close()

In [ ]:
# Connect to the newly created database
conn = psycopg2.connect(
    dbname=DB_NAME, user=USER, password=PASSWORD, host=HOST, port=PORT
)
cur = conn.cursor()

# Create the 'movies' table inside the 'parsee' schema
cur.execute("""
    CREATE SCHEMA IF NOT EXISTS parsee;
    CREATE TABLE IF NOT EXISTS parsee.movies (
        rank SERIAL PRIMARY KEY,
        year INT NOT NULL,
        title TEXT NOT NULL,
        gross TEXT NOT NULL,
        director TEXT NOT NULL,
        country TEXT NOT NULL
    );
""")

# Clear existing data before inserting new records
cur.execute("TRUNCATE TABLE parsee.movies RESTART IDENTITY;")

# Load data from CSV
with open("top_movies.csv", "r", newline="", encoding="utf-8") as csvfile:
    data = csv.reader(csvfile, delimiter=",")
    next(data)  # Skip header row

    cur.executemany(
        "INSERT INTO parsee.movies (title, year, gross, director, country) VALUES (%s, %s, %s, %s, %s);",
        data
    )

conn.commit()
print("Data successfully inserted into the database.")

cur.close()
conn.close()